In [1]:
import os
import sys
import time
from pathlib import Path

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

In [2]:
import pandas as pd
from PIL import Image, ImageDraw
import copy

In [5]:
from services.xml_document_info import (get_xml_info, get_xml_image_info)
from services.get_xml import  create_pdf_processing_paths, extract_pdf_metadata, process_input_pdf
from anuvaad_auditor.loghandler import log_info
from anuvaad_auditor.loghandler import log_error
from src.services import main
from src.services.get_underline import get_underline
from services import get_xml
from src.services.child_text_unify_to_parent import ChildTextUnify
from services.preprocess import prepocess_pdf_regions
from services.get_tables import page_num_correction , get_text_table_line_df
from src.services.ocr_text_utilities import  tesseract_ocr
from src.services.get_response import process_image_df,  process_table_df, df_to_json, process_line_df, process_bg_image

from utilities.filesystem import (create_directory)
import config

import src.utilities.app_context as app_context
app_context.init()

In [6]:
def draw_box(filepath, desired_width, desired_height, df, color="green", save=False):
    image  = Image.open(filepath)
    image  = image.resize((desired_width, desired_height))
    draw   = ImageDraw.Draw(image)
    
    for index, row in df.iterrows():
        left   = int(row['text_left'])
        right  = int(row['text_width'] + left)
        top    = int(row['text_top'])
        bottom = int(row["text_height"] + top)
        
        draw.rectangle(((left, top), (right,bottom)), outline=color)
    save_filepath = os.path.join(os.path.dirname(filepath), 'processed_' + os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

def show_df(df):
    return df.head(df.shape[0])

## start of the program

In [7]:
'''
  folder structure of test data goes like this
  - notebooks
      - sample-data
          - input
          - output
          
  the pdfs are present in "input" directory and they are *.pdf is added into .gitignore
  just to save repo size.
'''

base_dir   = os.getcwd()
input_dir  = os.path.join(base_dir, 'sample-data','input')
save_dir   = os.path.join(base_dir, 'sample-data', 'bbox_output')
output_dir = os.path.join(base_dir, 'sample-data', 'output')

#filename   = '6251_2016_3_1501_19387_Judgement_06-Jan-2020.pdf'
filename  = 'Madras_HC_02.pdf'


In [8]:
from src.services import main

In [ ]:
app_context.application_context = dict({'task_id': 'BM-DUMMY'})
pdf_path   = os.path.join(input_dir, filename)

data       = main.DocumentStructure(app_context, filename, "hi", base_dir=input_dir)

[2020-09-09 12:13:32,140] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing <module 'src.utilities.app_context' from '/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/app_context.py'>
[2020-09-09 12:13:32,142] {loghandler.py:20} MainThread INFO in loghandler: document preprocessing started ===>
[2020-09-09 12:13:32,153] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/Madras_HC_02_c6f0ca48-f267-11ea-b166-acde48001122
[2020-09-09 12:13:32,155] {loghandler.py:20} MainThread INFO in loghandler: filepath /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/Madras_HC_02.pdf, working_dir /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/Madras_H

/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[2020-09-09 12:13:35,702] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_hdfs completed in 0.5126080513000488/4, average per page 0.1281520128250122
[2020-09-09 12:13:35,862] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_vdfs completed in 0.15936803817749023/4, average per page 0.03984200954437256
[2020-09-09 12:13:36,318] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_hdfs completed in 0.45418715476989746/4, average per page 0.11354678869247437
[2020-09-09 12:13:36,328] {loghandler.py:20} MainThread INFO in loghandler: UnderLineDetection successfully completed


/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


[2020-09-09 12:13:36,476] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 0.1467418670654297/4, average per page 0.03668546676635742
[2020-09-09 12:13:36,477] {loghandler.py:20} MainThread INFO in loghandler: tesseract ocr started  ===>
/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/Madras_HC_02_c6f0ca48-f267-11ea-b166-acde48001122/images/Madras_HC_020001-1.jpg
 ]
 BEFORE THE MADURAI BENCH OF MADRAS HIGH COURT
 Malaisamy,
 Inspector of Police,
 Thallakulam Police Station,
 Thallakulam, Madurai.
 : Respondent
 PRAYER: Contempt Petition filed under Section व4 of the Contempt of Court Act, to punish the contemnor / respondent herein for committing grave contempt and gross disobedience of the order passed by this Court in Crl.0.P.(MD)No0.l0790 of 20l9 dated 3.07.20709.
 For Petitioner
 : Mr.M.M.Manivel Pandian
 DATED: 26.02.2020
 For Respondent
 : Mr.S.Chandrasekar,
 Additio

In [67]:
def draw_bbox_image(draw,page_data):
    
    if page_data['images']:
        try:
            for image_block in page_data['images']:
                    top     = image_block["text_top"];         left   = image_block["text_left"];  
                    bottom  = top+image_block["text_height"];  right  = left+image_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='green')
            return draw
        except:
            pass

In [68]:
def draw_bbox_table(draw,page_data):
    
    if page_data['tables']:
        try:
            for table_block in page_data['tables']:
                    top     = table_block["text_top"];         left   = table_block["text_left"];  
                    bottom  = top+table_block["text_height"];  right  = left+table_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='blue')
            return draw
        except:
            pass

In [69]:
def draw_bbox_text(draw,page_data):
    
    if page_data['text_blocks']:
        try:
            for text_block in page_data['text_blocks']:
                top     = text_block["text_top"];         left   = text_block["text_left"];  
                bottom  = top+text_block["text_height"];  right  = left+text_block["text_width"]
                draw.rectangle(((left, top), (right,bottom)), outline='red')
            return draw
        except:
            pass

In [70]:
def draw_bbox_pdf(data,image_files,save_path):
    for page_no in range(len(data)):
        image_path = sorted(image_files)[page_no]
        page_data = data[page_no]
        page_width = page_data['page_width']; page_height = page_data['page_height']
        image  = Image.open(image_path)
        image  = image.resize((page_width, page_height))
        draw   = ImageDraw.Draw(image)
        draw  = draw_bbox_text(draw,page_data)
        draw  = draw_bbox_image(draw,page_data)
        draw  = draw_bbox_table(draw,page_data)
        save_filepath = os.path.join(save_path,image_path.split('images/')[1])
        image.save(save_filepath)
            

In [71]:
draw_bbox_pdf(response,pdf_image_paths,save_path)

In [11]:
attr = ''

